In [2]:
import os
os.chdir("C:/Users/RAKESH MOURYA/Desktop/Bike_Sharing_Prediction/Bike_Sharing_Prediction")
%pwd

'C:\\Users\\RAKESH MOURYA\\Desktop\\Bike_Sharing_Prediction\\Bike_Sharing_Prediction'

In [1]:
# in entity/config_entity.py
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class TrainingConfig:
    root_dir: Path
    transformed_train_path: Path
    model_path: Path


In [3]:
from bike_rental.constants import*
from bike_rental.utils.common import read_yaml, create_directories

In [4]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])
    
    def get_training_config(self) -> TrainingConfig:
        config = self.config.training

        # Create training artifact directory
        create_directories([config.root_dir])

        training_config = TrainingConfig(
            root_dir=Path(config.root_dir),
            transformed_train_path=Path(config.transformed_train_path),
            model_path=Path(config.model_path)
        )

        return training_config

In [5]:
import os
import urllib.request as request
from bike_rental import logger
from bike_rental.utils.common import get_size

In [4]:
import pandas as pd
import pickle
import statsmodels.api as sm
from bike_rental import logger

class ModelTrainer:
    def __init__(self, config):
        self.config = config

    def train(self):
        logger.info("Reading preprocessed training data...")
        train = pd.read_csv(self.config.transformed_train_path)

        # Split features and target
        X_train = train.drop("cnt", axis=1)
        y_train = train["cnt"]

        # Final selected features from VIF analysis
        selected_features = [
            'yr', 'temp', 'holiday', 'Sep', 'Light Snow', 'Mist + Cloudy',
            'spring', 'summer', 'winter', 'Jan'
        ]

        X_train_sel = X_train[selected_features]

        # Add constant for OLS
        X_train_sm = sm.add_constant(X_train_sel).astype(float)

        logger.info("Training OLS model with selected features...")
        ols_model = sm.OLS(y_train, X_train_sm).fit()

        # Save trained model + features
        model_artifact = {
            "model": ols_model,
            "features": selected_features
        }
        with open(self.config.model_path, "wb") as f:
            pickle.dump(model_artifact, f)

        logger.info(f"Model trained and saved at {self.config.model_path}")
        logger.info(ols_model.summary())

        return ols_model, selected_features


In [7]:
try:
    config = ConfigurationManager()
    training_config = config.get_training_config()
    trainer = ModelTrainer(training_config)
    trainer.train()
except Exception as e:
    raise e
    
    

[2025-09-14 14:35:59,453: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-09-14 14:35:59,473: INFO: common: yaml file: params.yaml loaded successfully]
[2025-09-14 14:35:59,473: INFO: common: created directory at: artifacts]
[2025-09-14 14:35:59,482: INFO: common: created directory at: artifacts/training]
[2025-09-14 14:35:59,487: INFO: 2246508523: Reading preprocessed training data...]
[2025-09-14 14:35:59,619: INFO: 2246508523: Training OLS model with selected features...]
[2025-09-14 14:35:59,705: INFO: 2246508523: Model trained and saved at artifacts\training\model.pkl]
[2025-09-14 14:35:59,737: INFO: 2246508523:                             OLS Regression Results                            
Dep. Variable:                    cnt   R-squared:                       0.822
Model:                            OLS   Adj. R-squared:                  0.818
Method:                 Least Squares   F-statistic:                     230.9
Date:                Sun, 14 Sep 202